## Neste notebook, você codificará do zero seu primeiro agente de Reinforcement Learning jogando FrozenLake ❄️ usando Q-Learning

Adaptado HuggingFace

<img src="https://www.gymlibrary.dev/_images/frozen_lake.gif" alt="Environments"/>

###🎮 Environments:

>

- [FrozenLake-v1](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/)


###📚 RL-Library:

- Python and NumPy
- [Gym](https://www.gymlibrary.dev/)

## Pequena revisão de Q-Learning

- O *Q-Learning* **é o algoritmo RL que**

   - Treina *Q-Function*, uma **função ação-valor (action-value function)** que contém, como memória interna, uma *Q-table* **que contém todos os valores do par estado-ação.**

   - Dado um estado e uma ação, nossa Q-Function **pesquisará em sua Q-table o valor correspondente.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-function-2.jpg" alt="Q function"  width="100%"/>

- Quando o treinamento é concluído,**temos uma Função-Q ideal, portanto, uma Tabela-Q ideal.**

- E se **tivermos uma função Q ótima**,
ter uma política ideal, pois **sabemos para cada estado qual é a melhor ação a ser tomada.**

Mas, no começo, nossa **Q-Table é inútil, pois fornece um valor arbitrário para cada par estado-ação (na maioria das vezes, inicializamos a Q-Table com valores 0)**. Mas, conforme vamos explorando o ambiente e atualizando nosso Q-Table, ele nos dará aproximações cada vez melhores

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/q-learning.jpeg" alt="q-learning.jpeg" width="100%"/>

This is the Q-Learning pseudocode:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" alt="Q-Learning" width="100%"/>


# Let's code our first Reinforcement Learning algorithm 🚀

## Instalar dependências e criar um display virtual 🔽


In [ ]:
!pip install gym==0.24
!pip install pygame
!pip install numpy

!pip install pickle5
!pip install pyyaml==6.0
!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!pip install tqdm

In [ ]:
%%capture
!apt update
!apt install ffmpeg xvfb
!pip install xvfbwrapper
!pip install pyvirtualdisplay

Para garantir que as novas bibliotecas instaladas sejam usadas, **às vezes é necessário reiniciar o tempo de execução do notebook**. A próxima célula forçará o **tempo de execução a travar, então você precisará se conectar novamente e executar o código a partir daqui**.

In [ ]:
#import os
#os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Importação de pacotes 📦

Além das bibliotecas instaladas, utilizamos também:

- `random`: Para gerar números aleatórios (que serão úteis para a política epsilon-greedy).
- `imageio`: Para gerar um vídeo de replay.

In [ ]:
import numpy as np
import gym
import random
import imageio
import os
import tqdm

import pickle5 as pickle
from tqdm.notebook import tqdm

We're now ready to code our Q-Learning algorithm 🔥

# Part 1: Frozen Lake ⛄ (non slippery version)

## Criando o ambiente FrozenLake ⛄ (https://www.gymlibrary.dev/environments/toy_text/frozen_lake/)
---

💡 Um bom hábito quando você começa a usar um ambiente é verificar sua documentação

👉 https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

---

Vamos treinar nosso agente de Q-Learning **para navegar do estado inicial (S) para o estado objetivo (G) andando apenas em ladrilhos congelados (F) e evitando buracos (H)**.

Podemos ter dois tamanhos de ambiente:

- `map_name="4x4"`: uma versão de grade 4x4
- `map_name="8x8"`: uma versão em grade 8x8


Por enquanto vamos simplificar com o mapa 4x4 e antiderrapante (is_slippery=False)

In [ ]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)

### Vamos ver como fica o Environment:


In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

Vemos com a saída `Observation Space Shape Discrete(16)` que a observação é um inteiro representando a **posição atual do agente como current_row * nrows + current_col (onde tanto a linha quanto a coluna começam em 0)**.

Por exemplo, a posição do objetivo no mapa 4x4 pode ser calculada da seguinte forma: 3 * 4 + 3 = 15. O número de observações possíveis depende do tamanho do mapa. **Por exemplo, o mapa 4x4 tem 16 observações possíveis.**


Por exemplo, é assim que estado = 0 se parece:


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/frozenlake.png" alt="FrozenLake">

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

O espaço de ação (o conjunto de ações possíveis que o agente pode realizar) é discreto com 4 ações disponíveis 🎮:
- 0: VÁ PARA A ESQUERDA
- 1: DESCER
- 2: VÁ PARA A DIREITA
- 3: SUBIR

Função de recompensa 💰:
- Atingir meta: +1
- Furo de alcance: 0
- Alcance congelado: 0

## Criar e Inicializar a Q-table 🗄️
(👀 Step 1 of the pseudocode)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" alt="Q-Learning" width="100%"/>


É hora de inicializar nossa Q-table! Para saber quantas linhas (estados) e colunas (ações) usar, precisamos conhecer o espaço de ação e observação. Já conhecemos seus valores anteriormente, mas queremos obtê-los programaticamente para que nosso algoritmo generalize para diferentes ambientes. Gym nos fornece uma maneira de fazer isso: `env.action_space.n` e `env.observation_space.n`


In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

In [ ]:
# Vamos criar nossa Qtable de tamanho (state_space, action_space) e inicializar cada valor em 0 usando np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

In [ ]:
Qtable_frozenlake

## Defina a política gananciosa 🤖
Lembre-se de que temos duas políticas, pois o Q-Learning é um algoritmo **off-policy**. Isso significa que estamos usando uma **política diferente para atuar e atualizar a função de valor**.

- Política Epsilon-gananciosa (política de atuação)
- Greedy-policy (política de atualização)

A política gananciosa também será a política final que teremos quando o agente Q-learning for treinado. A política gulosa é usada para selecionar uma ação da tabela Q.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-4.jpg" alt="Q-Learning" width="100%"/>


In [ ]:
def greedy_policy(Qtable, state):
  # Exploitation
  action = np.argmax(Qtable[state][:])

  return action

##Defina a política gananciosa de epsilon 🤖

Epsilon-greedy é a política de treinamento que lida com a troca de exploração/exploração.

A ideia com epsilon-greedy:

- Com *probabilidade 1 - ɛ* : **fazemos exploitation** (ou seja, nosso agente seleciona a ação com o maior valor do par estado-ação).

- Com *probabilidade ɛ*: fazemos **exploration** (tentativa de ação aleatória).

E à medida que o treinamento avança, progressivamente **reduzimos o valor do epsilon, pois precisaremos de cada vez menos exploration e mais exploitation.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-4.jpg" alt="Q-Learning" width="100%"/>


In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Gera aleatoriamente um número entre 0 e 1
  random_int = random.uniform(0,1)
  # if random_int > maior que epsilon --> exploitation
  if random_int > epsilon:
     # Execute a ação com o maior valor dado um estado
     # np.argmax pode ser útil aqui
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

## Definindo os hiperparâmetros ⚙️
Os hiperparâmetros relacionados à exploração são alguns dos mais importantes.

- Precisamos garantir que nosso agente **explore o espaço de estados** o suficiente para aprender uma boa aproximação de valor. Para fazer isso, precisamos ter decaimento progressivo do epsilon.
- Se você diminuir o epsilon muito rápido (decay_rate muito alto), **você corre o risco de que seu agente fique preso**, já que seu agente não explorou o espaço de estado o suficiente e, portanto, não pode resolver o problema.

In [ ]:
# Parâmetros de treinamento
n_training_episodes = 10000 # Total de episódios de treinamento
learning_rate = 0.7 # Taxa de aprendizado

# Parâmetros de avaliação
n_eval_episodes = 100 # Número total de episódios de teste

# Parâmetros do ambiente
env_id = "FrozenLake-v1" # Nome do ambiente
max_steps = 99 # Max passos por episódio
gamma = 0.95 # Taxa de desconto
eval_seed = [] # A semente de avaliação do ambiente

# Parâmetros de exploração
max_epsilon = 1.0 # Probabilidade de exploração no início
min_epsilon = 0.05 # Probabilidade mínima de exploração
decay_rate = 0.0005 # Taxa de decaimento exponencial para prob de exploração

## Rotina de Treinamento

O loop de treinamento é assim:
```
Por episódio no total de episódios de treino:

Reduza o epsilon (já que precisamos cada vez menos de exploração)
Redefinir o ambiente

   Para passo em passos de tempo máximo:
     Escolha a ação At usar a política gananciosa do epsilon
     Tome a ação (a) e observe o(s) estado(s) resultante(s) e a recompensa (r)
     Atualize o valor Q Q(s,a) usando a equação de Bellman Q(s,a) + lr [R(s,a) + gama * max Q(s',a') - Q(s,a)]
     Se terminar, termine o episódio
     Nosso próximo estado é o novo estado
```

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # # Reduzir epsilon (porque precisamos cada vez menos exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Redefinir o ambiente
    state = env.reset()
    step = 0
    done = False

    # repete
    for step in range(max_steps):
      # Escolha a ação At para usar a política gananciosa (greedy policy) do epsilon
      action = epsilon_greedy_policy(Qtable, state, epsilon)


      new_state, reward, done, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      if done:
        break

      state = new_state
  return Qtable

## Treinando o agente Q-Learning 🏃

In [ ]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)

## Let's see what our Q-Learning table looks like now 👀

In [ ]:
Qtable_frozenlake

## Avaliação do Método 📝

- Definimos o método de avaliação que vamos usar para testar nosso agente Q-Learning.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
   Avalie o agente para episódios ``n_eval_episodes`` e retorne recompensa média e padrão de recompensa.
   :param env: O ambiente de avaliação
   :param n_eval_episodes: Número de episódios para avaliar o agente
   :param Q: A tabela Q
   :param seed: A matriz de sementes de avaliação (para taxi-v3)
   """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state = env.reset(seed=seed[episode])
    else:
      state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
      action = greedy_policy(Q, state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward

      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Avaliando nosso agente Q-Learning 📈

- Normalmente, você deve ter uma recompensa média de 1,0
- O **ambiente é relativamente fácil** já que o espaço de estados é muito pequeno (16). O que você pode tentar fazer é [substituí-lo pela versão escorregadia](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/), que introduz estocasticidade, tornando o ambiente mais complexo.

In [ ]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

#### Do not modify this code

In [ ]:
def record_video(env, Qtable, out_directory, fps=1):
  """
   Gerar um vídeo de replay do agente
   :param env
   :param Qtable: Qtable do nosso agente
   :param out_directory
   :param fps: quantos quadros por segundo (com taxi-v3 e frozenlake-v1 usamos 1)
   """
  images = []
  done = False
  state = env.reset(seed=random.randint(0,500))
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
    action = np.argmax(Qtable[state][:])
    state, reward, done, info = env.step(action) # Colocamos diretamente next_state = state para a lógica de gravação
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
video_path =  "replay.mp4"
record_video(env, Qtable_frozenlake, video_path, 0.5)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os


# Show video
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)